In [1]:
from ultralytics import YOLO
import pandas as pd
import os
import cv2

In [4]:
import torch

print("✅ CUDA 사용 가능:", torch.cuda.is_available())
print("🖥️ GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("💡 PyTorch 버전:", torch.__version__)


✅ CUDA 사용 가능: True
🖥️ GPU 이름: NVIDIA GeForce RTX 3070 Ti Laptop GPU
💡 PyTorch 버전: 2.4.1+cu121


In [3]:
import torch

if torch.backends.mps.is_available():
    print("MPS device is available.")
    device = torch.device("mps")
else:
    print("MPS device is not available.")
    device = torch.device("cpu")

print(f"Using device: {device}")

# 모델을 MPS 장치로 보낼 수 있습니다.
# model.to(device)

MPS device is not available.
Using device: cpu


# 욜로를 이용해서 키포인트 추출해 파일명,클래스,바운딩박스,키포인트 csv로 저장

In [ ]:
from ultralytics import YOLO
import os
import pandas as pd

# YOLO 모델 로드
model = YOLO("yolov8m-pose.pt")  # 사전 학습된 포즈 모델

def iou(box1, box2):
    """ box: [x_center, y_center, w, h], 정규화된 좌표 기준 IOU """
    x1_min = box1[0] - box1[2]/2
    y1_min = box1[1] - box1[3]/2
    x1_max = box1[0] + box1[2]/2
    y1_max = box1[1] + box1[3]/2

    x2_min = box2[0] - box2[2]/2
    y2_min = box2[1] - box2[3]/2
    x2_max = box2[0] + box2[2]/2
    y2_max = box2[1] + box2[3]/2

    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)

    inter_w = max(0, inter_xmax - inter_xmin)
    inter_h = max(0, inter_ymax - inter_ymin)
    inter_area = inter_w * inter_h

    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area > 0 else 0

def process_folder(split_name, base_path="roboflowfin", iou_threshold=0.5):
    label_dir = os.path.join(base_path, split_name, 'labels')
    image_dir = os.path.join(base_path, split_name, 'images')
    output_rows = []

    for label_file in os.listdir(label_dir):
        if not label_file.endswith(".txt"):
            continue

        # 이미지 확장자 동적 처리
        image_file = None
        for ext in [".jpg", ".png"]:
            potential_image = label_file.replace(".txt", ext)
            if os.path.exists(os.path.join(image_dir, potential_image)):
                image_file = potential_image
                break
        if not image_file:
            print(f"⚠️ {label_file}에 대응하는 이미지 파일 없음")
            continue

        image_path = os.path.join(image_dir, image_file)

        # 라벨 파일 읽기 및 비어 있는지 확인
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            if not lines or all(line.strip() == "" for line in lines):  # 라벨 파일이 비어 있는 경우
                print(f"⚠️ {label_file} 라벨 파일이 비어 있음, 건너뜀")
                continue

        # YOLOv8 Pose 추론
        results = model(image_path)
        pred_boxes = results[0].boxes.xywhn.cpu().tolist()  # 예측 bbox
        keypoints_list = results[0].keypoints.xyn.cpu().tolist()

        # 라벨 파일 처리
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:  # 라벨 형식이 잘못된 경우
                print(f"⚠️ {label_file}의 라벨 형식이 잘못됨: {line.strip()}")
                continue
            class_id = int(parts[0])  # 클래스 ID 그대로 사용
            bbox_gt = list(map(float, parts[1:5]))  # GT 바운딩박스

            # 가장 높은 IOU를 가진 예측 박스 찾기
            best_iou = 0
            best_kps = None
            for pred_box, kps in zip(pred_boxes, keypoints_list):
                iou_val = iou(bbox_gt, pred_box)
                if iou_val > best_iou and iou_val >= iou_threshold:
                    best_iou = iou_val
                    best_kps = kps

            # 키포인트 처리
            if best_kps:
                padded_kps = best_kps + [0.0] * (51 - len(best_kps))  # 17×3 = 51
                row = [image_file, class_id] + bbox_gt + padded_kps  # class_id 사용
                output_rows.append(row)
            else:
                print(f"⚠️ {image_file}에서 keypoint 추론 실패 또는 IOU {iou_threshold} 미만")

    # 행이 없으면 건너뜀
    if not output_rows:
        print(f"❌ {split_name}에 유효한 데이터 없음")
        return

    # 컬럼 구성
    expected_len = len(output_rows[0])
    columns = ['filename', 'class_id', 'x_center', 'y_center', 'width', 'height']
    num_kps = expected_len - 6
    for i in range(num_kps // 3):
        columns += [f'kp{i}_x', f'kp{i}_y', f'kp{i}_v']

    df = pd.DataFrame(output_rows, columns=columns)
    df.to_csv(f"{split_name}_pose_from_yolo.csv", index=False)
    print(f"✅ {split_name}_pose_from_yolo.csv 저장 완료 ({len(df)} rows)")

# 실행
for split in ['train', 'valid', 'test']:
    process_folder(split)


image 1/1 /Users/parkseongho/visualstudio-workspace/python-project/roboflowfin/train/images/IMG_9856_jpg.rf.ea556afa054f1d58a77368e0d6fbe27e.jpg: 640x640 2 persons, 192.6ms
Speed: 0.7ms preprocess, 192.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
⚠️ IMG_9856_jpg.rf.ea556afa054f1d58a77368e0d6fbe27e.jpg에서 keypoint 추론 실패 또는 IOU 0.5 미만

image 1/1 /Users/parkseongho/visualstudio-workspace/python-project/roboflowfin/train/images/-7-_JPG_jpg.rf.99ae399f205cb2cae89e9fecc260a512.jpg: 640x640 1 person, 214.1ms
Speed: 0.5ms preprocess, 214.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/parkseongho/visualstudio-workspace/python-project/roboflowfin/train/images/posture-757-_jpg.rf.25f4dc40ddf12b83fe039c6e0f2af371.jpg: 640x640 1 person, 212.1ms
Speed: 0.7ms preprocess, 212.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/parkseongho/visualstudio-workspace/python-project/roboflowfin/train/images/sit

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import cv2
import os
import ast

# CSV 로드
df = pd.read_csv("train_pose_from_yolo.csv")
image_dir = "roboflowfin/train/images"

# 시각화 루프
for idx, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    if not os.path.exists(img_path):
        continue

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # 바운딩박스 그리기
    x_center = row['x_center'] * w
    y_center = row['y_center'] * h
    box_width = row['width'] * w
    box_height = row['height'] * h
    x1 = int(x_center - box_width / 2)
    y1 = int(y_center - box_height / 2)
    x2 = int(x_center + box_width / 2)
    y2 = int(y_center + box_height / 2)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 키포인트 그리기
    for i in range(17):  # 17 keypoints 기준
        try:
            point_str = row[f'kp{i}_x']  # 예: "[0.123, 0.456]"
            if isinstance(point_str, str):
                x, y = ast.literal_eval(point_str)  # 문자열 → [x, y]
                x, y = float(x) * w, float(y) * h
                if x > 0 and y > 0:
                    cv2.circle(img, (int(x), int(y)), 4, (0, 0, 255), -1)
        except Exception as e:
            print(f"❌ keypoint {i} 처리 중 오류: {e}")

    out_path = os.path.join("visualized", row['filename'])
    os.makedirs("visualized", exist_ok=True)
    cv2.imwrite(out_path, img)


# 키포인트를 배열에서 꺼내서 형식을 바꿈

In [17]:
import pandas as pd
import ast

df = pd.read_csv("valid_pose_from_yolo.csv")

# 새로운 데이터프레임 구성
new_rows = []

for idx, row in df.iterrows():
    new_row = {
        'filename': row['filename'],
        'class_id': row['class_id'],
        'x_center': row['x_center'],
        'y_center': row['y_center'],
        'width': row['width'],
        'height': row['height'],
    }

    for i in range(17):
        try:
            value = row[f'kp{i}_x']
            if isinstance(value, str):
                xy = ast.literal_eval(value)  # "[x, y]" → [x, y]
            elif isinstance(value, float):  # 이미 float인 경우 (예: 0.0)
                xy = [0.0, 0.0]  # 누락된 keypoint로 간주
            else:
                raise ValueError("알 수 없는 타입")

            new_row[f'kp{i}_x'] = xy[0]
            new_row[f'kp{i}_y'] = xy[1]
            new_row[f'kp{i}_v'] = 0.0 if xy == [0.0, 0.0] else 2.0

        except Exception as e:
            print(f"⚠️ kp{i} 파싱 실패: {e}")
            new_row[f'kp{i}_x'] = 0.0
            new_row[f'kp{i}_y'] = 0.0
            new_row[f'kp{i}_v'] = 0.0


    new_rows.append(new_row)

df_new = pd.DataFrame(new_rows)
df_new.to_csv("valid_pose_parsed.csv", index=False)
print("✅ 새 구조의 CSV 저장 완료!")


✅ 새 구조의 CSV 저장 완료!


# 바운딩 박스와 키포인트를 이미지에 그림

In [25]:
import pandas as pd
import cv2
import os

# CSV 로드
df = pd.read_csv("train_pose_parsed.csv")  # 새 CSV 파일명으로 바꿔주세요
image_dir = "roboflowfin/train/images"

# 시각화 루프
for idx, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    if not os.path.exists(img_path):
        continue

    img = cv2.imread(img_path)
    # h, w = img.shape[:2]

    # # 바운딩박스 그리기
    # x_center = row['x_center'] * w
    # y_center = row['y_center'] * h
    # box_width = row['width'] * w
    # box_height = row['height'] * h
    # x1 = int(x_center - box_width / 2)
    # y1 = int(y_center - box_height / 2)
    # x2 = int(x_center + box_width / 2)
    # y2 = int(y_center + box_height / 2)
    # cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 키포인트 그리기
    for i in range(17):  # 17 keypoints
        try:
            x = float(row[f'kp{i}_x']) * w
            y = float(row[f'kp{i}_y']) * h
            v = float(row[f'kp{i}_v'])

            if v >= 1:  # 보이는 점만 빨간색으로
                cv2.circle(img, (int(x), int(y)), 4, (0, 0, 255), -1)
            elif v == 0:
                cv2.circle(img, (int(x), int(y)), 4, (128, 128, 128), -1)  # 안 보이는 점은 회색
        except Exception as e:
            print(f"⚠️ keypoint {i} 처리 오류: {e}")

    out_path = os.path.join("train-visualized", row['filename'])
    os.makedirs("train-visualized", exist_ok=True)
    cv2.imwrite(out_path, img)


# 스켈레톤 시각화

In [ ]:
import pandas as pd
import cv2
import os

# CSV 로드
df = pd.read_csv("train_pose_parsed.csv")
image_dir = "roboflowfin/train/images"

# COCO keypoint 연결 구조
skeleton = [
    (5, 7), (7, 9),    # 왼팔
    (6, 8), (8, 10),   # 오른팔
    (11, 13), (13, 15),  # 왼다리
    (12, 14), (14, 16),  # 오른다리
    (5, 6), (5, 11), (6, 12),  # 몸통
    (0, 1), (1, 3), (0, 2), (2, 4)  # 얼굴
]

for idx, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    if not os.path.exists(img_path):
        continue

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # 바운딩 박스
    x_center = row['x_center'] * w
    y_center = row['y_center'] * h
    box_width = row['width'] * w
    box_height = row['height'] * h
    x1 = int(x_center - box_width / 2)
    y1 = int(y_center - box_height / 2)
    x2 = int(x_center + box_width / 2)
    y2 = int(y_center + box_height / 2)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # keypoints 저장 (나중에 skeleton용)
    keypoints = []

    for i in range(17):
        try:
            x = float(row[f'kp{i}_x']) * w
            y = float(row[f'kp{i}_y']) * h
            v = float(row[f'kp{i}_v'])

            keypoints.append((x, y, v))  # 저장

            if v >= 1:
                cv2.circle(img, (int(x), int(y)), 4, (0, 0, 255), -1)  # 보이는 점: 빨간색
            elif v == 0:
                cv2.circle(img, (int(x), int(y)), 4, (128, 128, 128), -1)  # 안 보이는 점: 회색

        except Exception as e:
            print(f"⚠️ keypoint {i} 오류: {e}")
            keypoints.append((0.0, 0.0, 0.0))

    # 스켈레톤 선 그리기
    for i, j in skeleton:
        x1, y1, v1 = keypoints[i]
        x2, y2, v2 = keypoints[j]   
        if (x1 > 0 and y1 > 0) and (x2 > 0 and y2 > 0):
            cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)  # 노란색 선

    out_path = os.path.join("skeleton-visualized", row['filename'])
    os.makedirs("skeleton-visualized", exist_ok=True)
    cv2.imwrite(out_path, img)
